## jieba

#### “结巴”中文分词：做最好的 Python 中文分词组件

"Jieba" (Chinese for "to stutter") Chinese text segmentation: built to be the best Python Chinese word segmentation module.

Scroll down for English documentation.

### 特点

支持三种分词模式：

精确模式，试图将句子最精确地切开，适合文本分析；

全模式，把句子中所有的可以成词的词语都扫描出来, 速度非常快，但是不能解决歧义；

搜索引擎模式，在精确模式的基础上，对长词再次切分，提高召回率，适合用于搜索引擎分词。

支持繁体分词

支持自定义词典

MIT 授权协议

### 友情链接

https://github.com/baidu/lac  百度中文词法分析（分词+词性+专名）系统

https://github.com/baidu/AnyQ 百度FAQ自动问答系统

https://github.com/baidu/Senta 百度情感识别系统

### 安装说明

代码对 Python 2/3 均兼容

全自动安装：easy_install jieba 或者 pip install jieba / pip3 install jieba

半自动安装：先下载 http://pypi.python.org/pypi/jieba/ ，解压后运行 python setup.py install

手动安装：将 jieba 目录放置于当前目录或者 site-packages 目录

通过 import jieba 来引用

### 算法

基于前缀词典实现高效的词图扫描，生成句子中汉字所有可能成词情况所构成的有向无环图 (DAG)

采用了动态规划查找最大概率路径, 找出基于词频的最大切分组合

对于未登录词，采用了基于汉字成词能力的 HMM 模型，使用了 Viterbi 算法

### 主要功能

#### 1.分词

代码示例

In [4]:
# encoding=utf-8
import jieba

seg_list = jieba.cut("我来到北京清华大学", cut_all=True)
print("Full Mode: " + "/ ".join(seg_list))  # 全模式

seg_list = jieba.cut("我来到北京清华大学", cut_all=False)
print("Default Mode: " + "/ ".join(seg_list))  # 精确模式

seg_list = jieba.cut("他来到了网易杭研大厦")  # 默认是精确模式
print(", ".join(seg_list))

seg_list = jieba.cut_for_search("小明硕士毕业于中国科学院计算所，后在日本京都大学深造")  # 搜索引擎模式
print(", ".join(seg_list))

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\darius\AppData\Local\Temp\jieba.cache
Loading model cost 0.583 seconds.
Prefix dict has been built succesfully.


Full Mode: 我/ 来到/ 北京/ 清华/ 清华大学/ 华大/ 大学
Default Mode: 我/ 来到/ 北京/ 清华大学
他, 来到, 了, 网易, 杭研, 大厦
小明, 硕士, 毕业, 于, 中国, 科学, 学院, 科学院, 中国科学院, 计算, 计算所, ，, 后, 在, 日本, 京都, 大学, 日本京都大学, 深造


2.添加自定义词典

#### 载入词典

开发者可以指定自己自定义的词典，以便包含 jieba 词库里没有的词。虽然 jieba 有新词识别能力，但是自行添加新词可以保证更高的正确率

用法： jieba.load_userdict(file_name) # file_name 为文件类对象或自定义词典的路径

词典格式和 dict.txt 一样，一个词占一行；每一行分三部分：词语、词频（可省略）、词性（可省略），用空格隔开，顺序不可颠倒。file_name 若为路径或二进制方式打开的文件，则文件必须为 UTF-8 编码。

词频省略时使用自动计算的能保证分出该词的词频。

#### 例如：

更改分词器（默认为 jieba.dt）的 tmp_dir 和 cache_file 属性，可分别指定缓存文件所在的文件夹及其文件名，用于受限的文件系统。

范例：

自定义词典：https://github.com/fxsjy/jieba/blob/master/test/userdict.txt

用法示例：https://github.com/fxsjy/jieba/blob/master/test/test_userdict.py

之前： 李小福 / 是 / 创新 / 办 / 主任 / 也 / 是 / 云 / 计算 / 方面 / 的 / 专家 /

加载自定义词库后：　李小福 / 是 / 创新办 / 主任 / 也 / 是 / 云计算 / 方面 / 的 / 专家 /

### 调整词典

使用 add_word(word, freq=None, tag=None) 和 del_word(word) 可在程序中动态修改词典。

使用 suggest_freq(segment, tune=True) 可调节单个词语的词频，使其能（或不能）被分出来。

注意：自动计算的词频在使用 HMM 新词发现功能时可能无效。

代码示例：

In [6]:
 print('/'.join(jieba.cut('如果放到post中将出错。', HMM=False)))

如果/放到/post/中将/出错/。


In [7]:
jieba.suggest_freq(('中', '将'), True)

494

In [9]:
print('/'.join(jieba.cut('如果放到post中将出错。', HMM=False)))

如果/放到/post/中/将/出错/。


In [10]:
print('/'.join(jieba.cut('「台中」正确应该不会被切开', HMM=False)))

「/台/中/」/正确/应该/不会/被/切开


In [11]:
jieba.suggest_freq('台中', True)

69

In [12]:
print('/'.join(jieba.cut('「台中」正确应该不会被切开', HMM=False)))

「/台中/」/正确/应该/不会/被/切开


"通过用户自定义词典来增强歧义纠错能力" --- https://github.com/fxsjy/jieba/issues/14
关键词提取

3.关键词提取

### 基于 TF-IDF 算法的关键词抽取

In [13]:
import jieba.analyse

代码示例 （关键词提取）

https://github.com/fxsjy/jieba/blob/master/test/extract_tags.py

关键词提取所使用逆向文件频率（IDF）文本语料库可以切换成自定义语料库的路径

    用法： jieba.analyse.set_idf_path(file_name) # file_name为自定义语料库的路径
    自定义语料库示例：https://github.com/fxsjy/jieba/blob/master/extra_dict/idf.txt.big
    用法示例：https://github.com/fxsjy/jieba/blob/master/test/extract_tags_idfpath.py

关键词提取所使用停止词（Stop Words）文本语料库可以切换成自定义语料库的路径

    用法： jieba.analyse.set_stop_words(file_name) # file_name为自定义语料库的路径
    自定义语料库示例：https://github.com/fxsjy/jieba/blob/master/extra_dict/stop_words.txt
    用法示例：https://github.com/fxsjy/jieba/blob/master/test/extract_tags_stop_words.py

关键词一并返回关键词权重值示例

    用法示例：https://github.com/fxsjy/jieba/blob/master/test/extract_tags_with_weight.py

### 基于 TextRank 算法的关键词抽取

jieba.analyse.textrank(sentence, topK=20, withWeight=False, allowPOS=('ns', 'n', 'vn', 'v')) 直接使用，接口相同，注意默认过滤词性。

jieba.analyse.TextRank() 新建自定义 TextRank 实例

算法论文： TextRank: Bringing Order into Texts

基本思想:

    将待抽取关键词的文本进行分词
    以固定窗口大小(默认为5，通过span属性调整)，词之间的共现关系，构建图
    计算图中节点的PageRank，注意是无向带权图

使用示例

见 test/demo.py

4.词性标注

jieba.posseg.POSTokenizer(tokenizer=None) 新建自定义分词器，tokenizer 参数可指定内部使用的 jieba.Tokenizer 分词器。jieba.posseg.dt 为默认词性标注分词器。

标注句子分词后每个词的词性，采用和 ictclas 兼容的标记法。
用法示例

In [18]:
import jieba.posseg as pseg
words = pseg.cut("我爱北京天安门")
for word, flag in words:
    print('%s %s' % (word, flag))

我 r
爱 v
北京 ns
天安门 ns


5.并行分词

原理：将目标文本按行分隔后，把各行文本分配到多个 Python 进程并行分词，然后归并结果，从而获得分词速度的可观提升

基于 python 自带的 multiprocessing 模块，目前暂不支持 Windows

用法：

例子：https://github.com/fxsjy/jieba/blob/master/test/parallel/test_file.py

实验结果：在 4 核 3.4GHz Linux 机器上，对金庸全集进行精确分词，获得了 1MB/s 的速度，是单进程版的 3.3 倍。

注意：并行分词仅支持默认分词器 jieba.dt 和 jieba.posseg.dt。

6.Tokenize：返回词语在原文的起止位置

注意，输入参数只接受 unicode

默认模式：

In [22]:
result = jieba.tokenize(u'永和服装饰品有限公司')
for tk in result:
    print("word %s\t\t start: %d \t\t end:%d" % (tk[0],tk[1],tk[2]))

word 永和		 start: 0 		 end:2
word 服装		 start: 2 		 end:4
word 饰品		 start: 4 		 end:6
word 有限公司		 start: 6 		 end:10


搜索模式：

In [23]:
result = jieba.tokenize(u'永和服装饰品有限公司', mode='search')
for tk in result:
    print("word %s\t\t start: %d \t\t end:%d" % (tk[0],tk[1],tk[2]))

word 永和		 start: 0 		 end:2
word 服装		 start: 2 		 end:4
word 饰品		 start: 4 		 end:6
word 有限		 start: 6 		 end:8
word 公司		 start: 8 		 end:10
word 有限公司		 start: 6 		 end:10


7.ChineseAnalyzer for Whoosh 搜索引擎

引用： from jieba.analyse import ChineseAnalyzer

用法示例：https://github.com/fxsjy/jieba/blob/master/test/test_whoosh.py

8.命令行分词

使用示例：python -m jieba news.txt > cut_result.txt

命令行选项（翻译）：

--help 选项输出：

### 延迟加载机制

jieba 采用延迟加载，import jieba 和 jieba.Tokenizer() 不会立即触发词典的加载，一旦有必要才开始加载词典构建前缀字典。如果你想手工初始 jieba，也可以手动初始化。

In [24]:
import jieba
jieba.initialize()  # 手动初始化（可选）

在 0.28 之前的版本是不能指定主词典的路径的，有了延迟加载机制后，你可以改变主词典的路径:

### 其他词典

1，占用内存较小的词典文件 https://github.com/fxsjy/jieba/raw/master/extra_dict/dict.txt.small

支持繁体分词更好的词典文件 https://github.com/fxsjy/jieba/raw/master/extra_dict/dict.txt.big

2/下载你所需要的词典，然后覆盖 jieba/dict.txt 即可；或者用 jieba.set_dictionary('data/dict.txt.big')

### 其他语言实现

结巴分词 Java 版本
作者：piaolingxue 地址：https://github.com/huaban/jieba-analysis

结巴分词 C++ 版本
作者：yanyiwu 地址：https://github.com/yanyiwu/cppjieba

结巴分词 Node.js 版本
作者：yanyiwu 地址：https://github.com/yanyiwu/nodejieba

结巴分词 Erlang 版本
作者：falood 地址：https://github.com/falood/exjieba

结巴分词 R 版本
作者：qinwf 地址：https://github.com/qinwf/jiebaR

结巴分词 iOS 版本
作者：yanyiwu 地址：https://github.com/yanyiwu/iosjieba

结巴分词 PHP 版本
作者：fukuball 地址：https://github.com/fukuball/jieba-php

结巴分词 .NET(C#) 版本
作者：anderscui 地址：https://github.com/anderscui/jieba.NET/

结巴分词 Go 版本
作者: wangbin 地址: https://github.com/wangbin/jiebago
作者: yanyiwu 地址: https://github.com/yanyiwu/gojieba

结巴分词Android版本
作者 Dongliang.W 地址：https://github.com/452896915/jieba-android

### 系统集成

Solr: https://github.com/sing1ee/jieba-solr

### 分词速度

1.5 MB / Second in Full Mode

400 KB / Second in Default Mode

测试环境: Intel(R) Core(TM) i7-2600 CPU @ 3.4GHz；《围城》.txt